## Episode 6: Domain, custom actions and slots

In this episode we covered the reamining components of the Rasa dialogue management models - domain, custom actions and slots.

### Define a domain

Update the domain with all intents, entities, actions and template for the medicare locator assistant:

In [ ]:
domain_yml = """
intents:
  - greet
  - goodbye
  - affirm
  - deny
  - mood_great
  - mood_unhappy
  - inform
  - search_provider
  - thanks

entities:
  - location
  - facility_type

slots:
  location:
    type: text
  facility_type:
    type: text
  address:
    type: unfeaturized

actions:
- utter_greet
- utter_cheer_up
- utter_did_that_help
- utter_happy
- utter_goodbye
- utter_how_can_i_help
- utter_ask_location
- action_facility_search

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"

  utter_how_can_i_help:
  - text: "Hello. I am a medicare locator. I can help you find nearset hospital. How can I help?"
  - text: "Hi. My name is medicare locator. How can I help you today?"

  utter_ask_location:
  - text: "Can you provide your location please?"
  - text: "To find the nearest {facility_type} I need your address."
"""

%store domain_yml > domain.yml

### Implement the custom action

In the episode we also implemented a simple custom actions which looks for a specified facility based on user's preferences and sets the returne address as a slot.

In [ ]:
action_py = """

from typing import Any, Text, Dict, List

from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
from rasa_sdk.events import SlotSet


class ActionFacilitySearch(Action):

    def name(self) -> Text:
        return "action_facility_search"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        facility = tracker.get_slot("facility_type")
        address = "300 Hyde St, San Francisco"
        dispatcher.utter_message("Here is the address of the {}:{}".format(facility, address))

        return [SlotSet("address", address)]
        
"""        
%store action_py > actions.py

### Updating the stories
Training stories should be updated to reflect the SlotSet events which are executed in specific custom ations:


In [ ]:
stories_md = """
## hospital search happy path
* greet
  - utter_how_can_i_help
* search_provider{"facility_type":"hospital", "location": "San Francisco"}
  - action_facility_search
  - slot{"address":"300 Hyde St, San Francisco"}
* thanks
  - utter_goodbye

## hospital search + location
* greet
  - utter_how_can_i_help
* search_provider{"facility_type":"hospital"}
  - utter_ask_location
* inform{"location":"San Francisco"}
  - action_facility_search
  - slot{"address":"300 Hyde St, San Francisco"}
* thanks
  - utter_goodbye

## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye

"""        
%store stories_md > data/stories.md


### Train the model
With training data, domain, custom actions in place we can train and test our assistant to see how it works with NLU and dialogue management models in place.

In [ ]:
import nest_asyncio

nest_asyncio.apply()
print("Event loop ready.")

In [ ]:
import rasa

# define the project files
config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"

#train the example assistant
model_path = rasa.train(domain, config, [training_files], output)

Test the assistant:

In [ ]:
from rasa.jupyter import chat
chat(model_path)